\
            # 47. 接口设计：RESTful（资源、方法、状态码）（RESTful API Design）

            目标：把接口“做对且做稳定”：资源建模、HTTP 语义、错误格式、分页、幂等、版本化与兼容性。
本章提供一个可运行的标准库 HTTPServer 示例（本机 Todo API）。

            > 约定：Python 3.8；示例尽量只用标准库；代码块可直接运行（第三方依赖会做可选降级）。


## 前置知识

- HTTP 基础概念（请求/响应/头/体）
- JSON（序列化/反序列化）
- 异常处理


## 知识点地图

- 1. 资源建模：URI 设计（collection / item）
- 2. HTTP 方法语义：安全性与幂等性
- 3. 状态码与错误格式：让客户端可预测
- 4. 分页/过滤/排序：可扩展查询
- 5. 版本化与兼容性：不要轻易破坏客户端
- 6. 可观测性与安全：request_id、日志、鉴权（概念）
- 7. 可运行示例：标准库 Todo REST API（GET/POST/PUT/DELETE）


## 自检清单（学完打勾）

- [ ] 能把业务抽象成资源（collection/item）并设计 URI
- [ ] 知道 GET/POST/PUT/PATCH/DELETE 的语义与幂等性
- [ ] 能选对状态码并返回统一错误结构
- [ ] 会设计分页/过滤/排序与字段选择
- [ ] 知道鉴权、版本化、可观测性（request_id）的基本做法


In [ ]:
\
from pathlib import Path

ART = Path('_nb_artifacts')
ART.mkdir(exist_ok=True)
print('artifacts dir:', ART.resolve())


## 知识点 1：资源建模：URI 设计（collection / item）

REST 把业务抽象成“资源”：
- 列表资源（collection）：`/todos`
- 单个资源（item）：`/todos/{id}`

建议：
- URI 用名词而不是动词（动词用 HTTP 方法表达）。
- 层级表达归属关系：`/users/{id}/orders`。


## 知识点 2：HTTP 方法语义：安全性与幂等性

- GET：读取（安全、幂等）
- POST：创建/提交动作（非幂等，通常返回 201 + Location）
- PUT：整体替换（幂等）
- PATCH：局部更新（通常非幂等，但可以设计成幂等）
- DELETE：删除（幂等）

幂等的重要性：配合重试与网络超时，避免重复创建/扣款等事故。


## 知识点 3：状态码与错误格式：让客户端可预测

常见状态码：
- 200 OK：成功
- 201 Created：创建成功
- 204 No Content：成功但无响应体（常用于 DELETE）
- 400 Bad Request：参数错误
- 401 Unauthorized / 403 Forbidden：鉴权/权限
- 404 Not Found：资源不存在
- 409 Conflict：冲突（例如唯一约束）
- 429 Too Many Requests：限流
- 500/502/503：服务端/网关问题

建议统一错误结构（示例）：
```json
{ "error": {"code": "INVALID_ARGUMENT", "message": "...", "details": {...}, "request_id": "..."} }
```


## 知识点 4：分页/过滤/排序：可扩展查询

常见分页：
- offset 分页：`?page=1&page_size=20`（简单，但深分页慢）
- cursor 分页：`?cursor=...&limit=20`（更稳定，推荐）

过滤与排序：
- `?status=done&sort=-created_at`
- 字段选择：`?fields=id,title,done`（减少传输）


## 知识点 5：版本化与兼容性：不要轻易破坏客户端

版本化方式：
- URL：`/v1/todos`（直观）
- Header：`Accept: application/vnd.xxx+json;version=1`

兼容性原则：
- 尽量“新增字段”而不是改语义/删字段
- 对未知字段保持忽略（客户端更健壮）


## 知识点 6：可观测性与安全：request_id、日志、鉴权（概念）

- request_id：每个请求生成/透传，便于排查（可从网关传入）。
- 日志：记录关键字段与耗时；避免记录敏感信息。
- 鉴权：JWT/Session/OAuth2；授权（RBAC/ABAC）。
- 限流与防刷：429 + 重试提示（Retry-After）。


## 知识点 7：可运行示例：标准库 Todo REST API（GET/POST/PUT/DELETE）

下面示例：
- 启动本机 HTTPServer（随机端口）
- 提供 /todos 与 /todos/{id}
- 客户端用 urllib 发请求验证行为

这不是生产框架，但能让你完整理解“方法/状态码/JSON/错误格式”的组合。


In [ ]:
import json
import threading
import uuid
from http.server import BaseHTTPRequestHandler, HTTPServer
from urllib.request import Request, urlopen


def json_resp(handler: BaseHTTPRequestHandler, status: int, payload=None, headers=None):
    body = b'' if payload is None else json.dumps(payload, ensure_ascii=False).encode('utf-8')
    handler.send_response(status)
    handler.send_header('Content-Type', 'application/json; charset=utf-8')
    handler.send_header('X-Request-Id', handler.request_id)
    if headers:
        for k, v in headers.items():
            handler.send_header(k, v)
    if body:
        handler.send_header('Content-Length', str(len(body)))
    handler.end_headers()
    if body:
        handler.wfile.write(body)


def parse_json(handler: BaseHTTPRequestHandler):
    n = int(handler.headers.get('Content-Length', '0') or '0')
    if n <= 0:
        return None
    raw = handler.rfile.read(n)
    return json.loads(raw.decode('utf-8'))


class TodoAPI(BaseHTTPRequestHandler):
    todos = {}

    def setup(self):
        super().setup()
        self.request_id = self.headers.get('X-Request-Id') or uuid.uuid4().hex

    def log_message(self, fmt, *args):
        return

    def do_GET(self):
        if self.path == '/todos':
            items = list(self.todos.values())
            return json_resp(self, 200, {'items': items})
        if self.path.startswith('/todos/'):
            todo_id = self.path.split('/', 2)[2]
            t = self.todos.get(todo_id)
            if not t:
                return json_resp(self, 404, {'error': {'code': 'NOT_FOUND', 'message': 'todo not found', 'request_id': self.request_id}})
            return json_resp(self, 200, t)
        return json_resp(self, 404, {'error': {'code': 'NOT_FOUND', 'message': 'path not found', 'request_id': self.request_id}})

    def do_POST(self):
        if self.path != '/todos':
            return json_resp(self, 404, {'error': {'code': 'NOT_FOUND', 'message': 'path not found', 'request_id': self.request_id}})
        try:
            data = parse_json(self) or {}
            title = (data.get('title') or '').strip()
            if not title:
                return json_resp(self, 400, {'error': {'code': 'INVALID_ARGUMENT', 'message': 'title required', 'request_id': self.request_id}})
            todo_id = uuid.uuid4().hex[:8]
            todo = {'id': todo_id, 'title': title, 'done': False}
            self.todos[todo_id] = todo
            return json_resp(self, 201, todo, headers={'Location': f'/todos/{todo_id}'})
        except Exception as e:
            return json_resp(self, 400, {'error': {'code': 'BAD_JSON', 'message': str(e), 'request_id': self.request_id}})

    def do_PUT(self):
        if not self.path.startswith('/todos/'):
            return json_resp(self, 404, {'error': {'code': 'NOT_FOUND', 'message': 'path not found', 'request_id': self.request_id}})
        todo_id = self.path.split('/', 2)[2]
        if todo_id not in self.todos:
            return json_resp(self, 404, {'error': {'code': 'NOT_FOUND', 'message': 'todo not found', 'request_id': self.request_id}})
        try:
            data = parse_json(self) or {}
            title = (data.get('title') or '').strip()
            done = bool(data.get('done'))
            if not title:
                return json_resp(self, 400, {'error': {'code': 'INVALID_ARGUMENT', 'message': 'title required', 'request_id': self.request_id}})
            todo = {'id': todo_id, 'title': title, 'done': done}
            self.todos[todo_id] = todo
            return json_resp(self, 200, todo)
        except Exception as e:
            return json_resp(self, 400, {'error': {'code': 'BAD_JSON', 'message': str(e), 'request_id': self.request_id}})

    def do_DELETE(self):
        if not self.path.startswith('/todos/'):
            return json_resp(self, 404, {'error': {'code': 'NOT_FOUND', 'message': 'path not found', 'request_id': self.request_id}})
        todo_id = self.path.split('/', 2)[2]
        self.todos.pop(todo_id, None)
        return json_resp(self, 204, None)


server = HTTPServer(('127.0.0.1', 0), TodoAPI)
host, port = server.server_address
threading.Thread(target=server.serve_forever, daemon=True).start()
base = f'http://{host}:{port}'


def call(method, path, payload=None):
    data = None if payload is None else json.dumps(payload).encode('utf-8')
    req = Request(base + path, method=method, data=data)
    req.add_header('Content-Type', 'application/json')
    req.add_header('X-Request-Id', 'demo-request-id')
    try:
        with urlopen(req, timeout=2) as resp:
            body = resp.read()
            return resp.status, resp.headers.get('Location'), body.decode('utf-8') if body else ''
    except Exception as e:
        # urllib 对 4xx/5xx 会抛异常，这里简化为打印
        return 'ERR', None, str(e)


print(call('POST', '/todos', {'title': 'learn restful'}))
print(call('GET', '/todos'))
# 取第一条的 id 再更新
status, _loc, body = call('GET', '/todos')
items = json.loads(body)['items']
first_id = items[0]['id']
print(call('PUT', f'/todos/{first_id}', {'title': 'learn restful (updated)', 'done': True}))
print(call('GET', f'/todos/{first_id}'))
print(call('DELETE', f'/todos/{first_id}'))
print(call('GET', '/todos'))

server.shutdown()


## 常见坑

- URI 写动词：/getTodo、/createTodo（不 RESTful）
- 状态码乱用：任何错误都返回 200
- 错误结构不统一：客户端难处理
- 不考虑幂等与重试：网络抖动导致重复创建
- 分页用 offset 深分页：性能变差且数据漂移


## 综合小案例：为你的业务设计一份 API 规范（API Spec）

选择一个熟悉的业务（例如：订单、文章、评论），写一份 API 规范：
- 资源与 URI
- 方法与状态码
- 请求/响应 JSON（含错误结构）
- 分页/过滤/排序
- 幂等策略（幂等键/唯一约束）


In [ ]:
# 建议用 Markdown 写一份 API Spec（也可以用 OpenAPI/Swagger）
# 本 cell 不运行代码。


## 自测题（不写代码也能回答）

- GET/POST/PUT/DELETE 的幂等性分别如何？为什么这很重要？
- 为什么建议统一错误结构？哪些字段最关键？
- cursor 分页比 offset 分页解决了什么问题？
- 版本化有哪些方式？各自优缺点是什么？


## 练习题（建议写代码）

- 为 POST /todos 增加幂等键（Idempotency-Key）并实现“重复请求不重复创建”（可用内存 dict 模拟）。
- 为 GET /todos 增加分页参数 page/page_size（offset 版）并实现。
- 把示例 API 的数据存储从内存换成 sqlite3（与 MySQL 章呼应）。
